In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing

2023-04-23 17:09:28.913207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 17:09:30.978198: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:$LD_LIBRARY_PATH
2023-04-23 17:09:30.978263: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:$LD_LIBRARY_PATH
2023-04-23 17:09:30.978268: W tensorflow/compiler/tf

In [2]:
# Bypass issues with invoking notebook with server arguments
sys.argv.clear()
sys.argv.append("")

In [3]:
%run utils/attention_graph.py
%run utils/mlflow_query.py
%run utils/loading.py
%run utils/comparison.py
%run utils/ranks.py

In [4]:
huawei_experiment_id = "519815984018260895"


In [6]:
mlflow_helper = MlflowHelper(pkl_file=Path("mlflow_run_df.pkl"), experiment_id=huawei_experiment_id)
mlflow_helper.query_valid_runs(pkl_file=Path("mlflow_run_df.pkl"))

Initialized with 350 MLFlow runs from pkl


Querying data per run...: 0it [00:00, ?it/s]


Queried 350 runs from MLFlow for mimic


Querying data per run...: 100%|██████████| 254/254 [00:00<00:00, 3444.52it/s]


Queried 350 runs from MLFlow for huawei_logs


Querying data per run...: 0it [00:00, ?it/s]


Queried 350 runs from MLFlow for hdfs


Querying data per run...: 0it [00:00, ?it/s]

Queried 350 runs from MLFlow for tbird_logs


In [7]:
huawei_df_all = mlflow_helper.huawei_run_df(
    valid_x_columns=["fine_log_cluster_template_drain", "coarse_log_cluster_template_drain", "", "medium_log_cluster_template_drain", "fine_log_cluster_template_spell", "coarse_log_cluster_template_spell",
                     "medium_log_cluster_template_spell", "fine_log_cluster_template_nulog", "coarse_log_cluster_template_nulog", "medium_log_cluster_template_nulog"],
    valid_y_columns=["attributes"],
)
huawei_df_all = huawei_df_all[
    huawei_df_all["data_params_ModelConfigbase_feature_embeddings_trainable"].astype(str) == "False"
    ]

huawei_df_all.groupby(by=[
    "data_params_SequenceConfigx_sequence_column_name",
    "data_params_SequenceConfigy_sequence_column_name",
    "data_tags_model_type",
]).agg({
    "info_run_id": len
})

,,,info_run_id
data_params_SequenceConfigx_sequence_column_name,data_params_SequenceConfigy_sequence_column_name,data_tags_model_type,
,attributes,simple,40
coarse_log_cluster_template_drain,attributes,simple,16
coarse_log_cluster_template_nulog,attributes,simple,15
coarse_log_cluster_template_spell,attributes,simple,15
fine_log_cluster_template_drain,attributes,simple,16
fine_log_cluster_template_nulog,attributes,simple,16
fine_log_cluster_template_spell,attributes,simple,15
medium_log_cluster_template_drain,attributes,simple,16
medium_log_cluster_template_nulog,attributes,simple,15


In [8]:
huawei_df = mlflow_helper.huawei_run_df(
    valid_x_columns=["fine_log_cluster_template", "medium_log_cluster_template", "coarse_log_cluster_template", ""],
    valid_y_columns=["attributes"],
)
huawei_df = huawei_df[
    huawei_df["data_params_ModelConfigbase_feature_embeddings_trainable"].astype(str) == "False"
    ]

huawei_df.groupby(by=[
    "data_params_SequenceConfigx_sequence_column_name",
    "data_params_SequenceConfigy_sequence_column_name",
    "data_tags_model_type",
]).agg({
    "info_run_id": len
})

,,,info_run_id
data_params_SequenceConfigx_sequence_column_name,data_params_SequenceConfigy_sequence_column_name,data_tags_model_type,
,attributes,simple,40
coarse_log_cluster_template,attributes,simple,25
fine_log_cluster_template,attributes,simple,25
medium_log_cluster_template,attributes,simple,25


In [9]:
huawei_metrics_df = mlflow_helper.load_best_metrics_for_ids(set(huawei_df["info_run_id"]))
huawei_metrics_df_all = mlflow_helper.load_best_metrics_for_ids(set(huawei_df_all["info_run_id"]))

Querying metrics for runs: 100%|██████████| 179/179 [00:10<00:00, 16.45it/s]


In [12]:
drain_run_id = huawei_df["info_run_id"].iloc[0]
drain_run_id

'f9ef674346eb4ba28a46a4d9b9c210f7'

# Functions

In [10]:
def calculate_shared_attention_weights(attention_weights: Dict[str, Dict[str, float]]):
    if attention_weights is None:
        return [0.0]
    attention_importances = calculate_attention_importances(attention_weights)
    shared_weights = [
        sum([
            float(weight) for con_feature, weight in attention_weights[in_feature].items()
            if len(attention_importances[con_feature]) > 1
        ])
        for in_feature in attention_weights
    ]
    return shared_weights

In [13]:
feature_node_mapping = create_graph_visualization(
    run_id=drain_run_id,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir,
    threshold=0.2,
    run_name='drain_hierarchy',
    use_node_mapping=False)

No attention file for run f9ef674346eb4ba28a46a4d9b9c210f7 in local MlFlow dir
